# Projeto SuperComputação 2024.2

### Nome: Luca Mizrahi

#### *Código-fonte de geração de grafos*

In [3]:
import networkx as nx
import random

# Parâmetros
num_vertices = 50  # Número de vértices no grafo
probabilidade_conexao = 0.7  # Probabilidade de haver uma aresta entre dois vértices (ajuste conforme necessário)

# Crie um grafo aleatório densamente conectado
grafo = nx.fast_gnp_random_graph(num_vertices, probabilidade_conexao)

# Nome do arquivo de saída
nome_arquivo = "grafo50.txt"

# Abra o arquivo para escrita
with open(nome_arquivo, 'w') as arquivo:
    # Escreva a quantidade de vértices e número de arestas na primeira linha
    arquivo.write(f"{num_vertices} {grafo.number_of_edges()}\n")

    # Escreva as arestas no formato de lista de adjacência
    for aresta in grafo.edges():
        arquivo.write(f"{aresta[0]+1} {aresta[1]+1}\n")  # +1 para ajustar os índices (começando em 1)

print(f"Grafo densamente conectado gerado e salvo em '{nome_arquivo}'.")

Grafo densamente conectado gerado e salvo em 'grafo50.txt'.


#### *Código para validação dos resultado*

In [ ]:
# Nome do arquivo de entrada
nome_arquivo = "grafo.txt"

# Abrir o arquivo e pular a primeira linha
with open(nome_arquivo, 'r') as arquivo:
    next(arquivo)  # Pula a primeira linha

    # Lê o grafo a partir das linhas restantes
    G = nx.parse_adjlist(arquivo)

# Encontrar todas as cliques maximais
cliques_maximais = list(nx.find_cliques(G))

# Encontrar a clique máxima (a maior)
clique_maxima = max(cliques_maximais, key=len)

print("Cliques maximais encontradas:")
for clique in cliques_maximais:
    print(clique)

print("Clique máxima encontrada:", clique_maxima)

-------------

#### *Etapa 1: Abordagem Exaustiva*

#### *Código C++ usando busca exaustiva para encontrar a clique máxima*

```cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <algorithm>

// Função para ler o grafo a partir do arquivo de entrada
std::vector<std::vector<int>> LerGrafo(const std::string& nomeArquivo, int& numVertices) {
    std::ifstream arquivo(nomeArquivo);
    int numArestas;
    arquivo >> numVertices >> numArestas;

    // Inicializa a matriz de adjacência com zeros
    std::vector<std::vector<int>> grafo(numVertices, std::vector<int>(numVertices, 0));

    // Lê as arestas e preenche a matriz de adjacência
    for (int i = 0; i < numArestas; ++i) {
        int u, v;
        arquivo >> u >> v;
        grafo[u - 1][v - 1] = 1;
        grafo[v - 1][u - 1] = 1;  // O grafo é não direcionado
    }

    arquivo.close();
    return grafo;
}

// Função para encontrar a clique máxima usando abordagem exaustiva
void EncontrarCliqueMaxima(const std::vector<std::vector<int>>& grafo, int numVertices, std::vector<int>& cliqueMaxima) {
    // Inicializa os candidatos com todos os vértices do grafo
    std::vector<int> candidatos(numVertices);
    for (int i = 0; i < numVertices; ++i) {
        candidatos[i] = i;
    }

    // Vetor auxiliar para a clique atual
    std::vector<int> cliqueAtual;

    // Função lambda recursiva para encontrar cliques
    std::function<void(std::vector<int>&, std::vector<int>&)> EncontrarCliques;
    EncontrarCliques = [&](std::vector<int>& cliqueAtual, std::vector<int>& candidatos) {
        for (size_t i = 0; i < candidatos.size(); ++i) {
            int v = candidatos[i];
            std::vector<int> novosCandidatos;

            // Filtra os candidatos para aqueles que são adjacentes ao vértice atual v
            for (size_t j = i + 1; j < candidatos.size(); ++j) {
                int u = candidatos[j];
                if (grafo[v][u]) {
                    novosCandidatos.push_back(u);
                }
            }

            // Adiciona o vértice atual à clique
            cliqueAtual.push_back(v);

            // Atualiza a clique máxima se a clique atual for maior
            if (cliqueAtual.size() > cliqueMaxima.size()) {
                cliqueMaxima = cliqueAtual;
            }

            // Chama recursivamente para explorar com os novos candidatos
            if (!novosCandidatos.empty()) {
                EncontrarCliques(cliqueAtual, novosCandidatos);
            }

            // Remove o vértice da clique atual ao voltar na recursão
            cliqueAtual.pop_back();
        }
    };

    // Inicia a busca pela clique máxima
    EncontrarCliques(cliqueAtual, candidatos);
}

int main() {
    int numVertices;
    std::string nomeArquivo = "grafo.txt";
    
    // Lê o grafo do arquivo e armazena como matriz de adjacência
    std::vector<std::vector<int>> grafo = LerGrafo(nomeArquivo, numVertices);

    // Vetor para armazenar a clique máxima encontrada
    std::vector<int> cliqueMaxima;
    
    // Encontra a clique máxima
    EncontrarCliqueMaxima(grafo, numVertices, cliqueMaxima);

    // Exibe o resultado
    std::cout << "Tamanho da clique máxima: " << cliqueMaxima.size() << std::endl;
    std::cout << "Vértices da clique máxima: ";
    for (int v : cliqueMaxima) {
        std::cout << (v + 1) << " "; // Exibindo os vértices na notação 1-baseada
    }
    std::cout << std::endl;

    return 0;
}
```

#### *Aproveite para pensar se é possível adotar alguma heurística. Por exemplo, ordenar os nós em função do grau de adjacência, ajuda?*

Sim, ordenar os nós pelo grau de adjacência pode ajudar. Isso se deve ao fato de que vértices com graus mais altos têm mais conexões, aumentando a probabilidade de formar cliques maiores. Ordenando os nós em ordem decrescente de grau, o algoritmo pode priorizar nós mais "conectados", encontrando cliques maiores mais rapidamente. Essa abordagem ajuda na **heurística gulosa**, onde tentamos primeiro explorar os vértices mais conectados para aumentar a clique.

**Implementação da Heurística no Pseudo-Código:**

```cpp
Função OrdenarPorGrau(grafo, numVertices):
    graus = []
    Para i de 0 até numVertices - 1:
        grau = ContarArestas(grafo[i])  # Conta o número de conexões (aresta) de cada nó
        Adicione (i, grau) em graus  # Adiciona o vértice e seu grau como um par
    Fim Para
    
    Ordene graus em ordem decrescente de grau
    candidatosOrdenados = [par[0] para cada par em graus]  # Extrai os vértices ordenados
    Retorne candidatosOrdenados
Fim Função
```

#### *Até qual tamanho de problema você conseguiu obter um resultado em tempo hábil (aprox. 15 min)? Você deve apresentar um pseudo-codigo de uma heurística para otimizar essa tarefa exaustiva. É possível implementar alguma poda? Algum critério que evite calcular um nó, dado que você já descobriu uma clique maior?*

Para o método exaustivo, o tempo de execução cresce exponencialmente conforme o número de vértices. Em muitos testes, um grafo com até **20-30 vértices** e grau moderado (número de arestas) pode ser resolvido em menos de 15 minutos, dependendo do hardware e das conexões do grafo. Acima disso, a abordagem exaustiva tende a ser inviável, especialmente para grafos densamente conectados.

O primeiro teste foi realizado com o grafo com os parâmetros padrão do código-fonte fornecido, que possui 1000 vértices e uma probabilidade de conexão de 0.7. Por conta desses parâmetros elevados a abordagem de busca exaustiva se tornou inviável, pois o tempo de execução foi muito alto, como mostrado na foto abaixo:

![Testes](imgs/teste_exaustiva1000.png)

Esse tempo de execução mostra o momento em que o código foi interrompido, pois o tempo de execução estava muito alto.


#### 3. Pseudo-código de Heurística para Otimizar a Tarefa Exaustiva

Uma heurística para otimizar a abordagem exaustiva é **usar poda** e **critério de parada precoce**:

- **Poda**: interrompe a exploração de uma possível clique se não for possível superá-la em tamanho em relação a uma clique máxima já encontrada.
- **Critério de parada**: se, ao explorar nós conectados a um subconjunto atual, descobrirmos que é impossível encontrar uma clique maior do que a já descoberta, abandonamos aquela linha de exploração.

#### Pseudo-Código com Poda e Critério de Parada:

```cpp
Função EncontrarCliqueMaximaOtimizada(grafo, numVertices):
    cliqueMaxima = ListaVazia()
    candidatos = OrdenarPorGrau(grafo, numVertices)

    Função Recursiva EncontrarCliques(cliqueAtual, candidatos):
        Se tamanho(cliqueAtual) + tamanho(candidatos) <= tamanho(cliqueMaxima):
            Retorne  # Poda: se não há potencial para clique maior, interrompe a recursão

        Para cada v em candidatos:
            novosCandidatos = [u para cada u em candidatos se grafo[v][u] == 1]  # Filtra adjacentes
            cliqueAtual.append(v)

            Se tamanho(cliqueAtual) > tamanho(cliqueMaxima):
                cliqueMaxima = copia(cliqueAtual)  # Atualiza clique máxima

            Se novosCandidatos não estiver vazio:
                EncontrarCliques(cliqueAtual, novosCandidatos)

            cliqueAtual.pop_back()  # Remove v ao voltar na recursão
    Fim Função

    EncontrarCliques(ListaVazia(), candidatos)
    Retorne cliqueMaxima
Fim Função
```

Isso ajuda a reduzir o número de combinações e, em muitos casos, melhora o tempo de execução.

Essas otimizações ajudarão a abordar o problema com maior eficiência, especialmente na abordagem exaustiva, permitindo resolver instâncias um pouco maiores dentro de um limite de tempo razoável.

-------------

#### *Etapa 2: Implementação com Threads OpenMP*

#### *Código C++ usando OpenMP para encontrar a clique máxima*

Abordagem Exaustiva:
Tamanho da clique máxima: 4
Vértices da clique máxima: 12 13 14 15 
Tempo de execução: 8.65e-06 segundos
 
Abordagem com OpenMP:
Tamanho da clique máxima: 4
Vértices da clique máxima: 12 13 14 15 
Tempo de execução: 3.8029e-05 segundos